In [ ]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
from sklearn.model_selection import train_test_split
import openml

# Load balance-scale dataset
dataset = openml.datasets.get_dataset(11)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)


print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))


In [ ]:
X_train.head()

In [ ]:
import category_encoders as ce
import numpy as np
import sklearn

low_cardinality_indices = []
high_cardinality_indices = []

categorical_feature_indices = []
for column_index in range(X_train.shape[1]):
    if categorical_indicator[column_index]:
        categorical_feature_indices.append(column_index)
        if len(X_train.iloc[:,column_index].unique()) < 10:
            low_cardinality_indices.append(X_train.columns[column_index])
        else:
            high_cardinality_indices.append(X_train.columns[column_index])
        
X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

encoder = ce.OrdinalEncoder(cols=X_train_raw.columns[categorical_feature_indices])
encoder.fit(X_train_raw)
X_train_raw = encoder.transform(X_train_raw)
X_valid_raw = encoder.transform(X_valid_raw)
X_test_raw = encoder.transform(X_test_raw)

median = X_train_raw.median(axis=0)
X_train_raw = X_train_raw.fillna(median)
X_valid_raw = X_valid_raw.fillna(median)
X_test_raw = X_test_raw.fillna(median)

y_train = y_train.values.codes.astype(np.float64)
y_valid = y_valid.values.codes.astype(np.float64)
y_test = y_test.values.codes.astype(np.float64)




In [ ]:
from GRANDE import GRANDE

params = {
        'depth': 5,
        'n_estimators': 2048,

        'learning_rate_weights': 0.005,
        'learning_rate_index': 0.01,
        'learning_rate_values': 0.01,
        'learning_rate_leaf': 0.01,

        'optimizer': 'SWA',
        'cosine_decay_steps': 0,

        'initializer': 'RandomNormal',

        'loss': 'crossentropy',
        'focal_loss': False,

        'from_logits': True,
        'apply_class_balancing': True,

        'dropout': 0.0,

        'selected_variables': 0.8,
        'data_subset_fraction': 1.0,
}

args = {
    'epochs': 1_000,
    'early_stopping_epochs': 25,
    'batch_size': 64,

    'cat_idx': categorical_feature_indices,
    'objective': 'classification',

    'metrics': ['F1'], # F1, Accuracy, R2
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train_raw,
          y_train=y_train,
          X_val=X_valid_raw,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test_raw)

In [ ]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    #class_weights = class_weights/sum(class_weights)#    
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    #sample_weights = sample_weights/sum(class_weights)
    return sample_weights

In [ ]:

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

quantile_noise = 1e-4
quantile_train = np.copy(X_train.values).astype(np.float64)
np.random.seed(42)
stds = np.std(quantile_train, axis=0, keepdims=True)
noise_std = quantile_noise / np.maximum(stds, quantile_noise)
quantile_train += noise_std * np.random.randn(*quantile_train.shape)       

scaler = sklearn.preprocessing.QuantileTransformer(output_distribution='normal')
scaler.fit(quantile_train)

X_train = scaler.transform(X_train.values.astype(np.float64))
X_valid = scaler.transform(X_valid.values.astype(np.float64))
X_test = scaler.transform(X_test.values.astype(np.float64))

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )

preds_xgb = model_xgb.predict_proba(X_test)

In [ ]:
from catboost import CatBoostClassifier, Pool

model_catboost = CatBoostClassifier(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

preds_catboost = model_catboost.predict_proba(X_test_raw)

In [ ]:
accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro')

print('Accuracy GRANDE:', accuracy)
print('F1 Score GRANDE:', f1_score)
print('ROC AUC GRANDE:', roc_auc)
print('\n')

accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro')

print('Accuracy XGB:', accuracy)
print('F1 Score XGB:', f1_score)
print('ROC AUC XGB:', roc_auc)
print('\n')

accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro')

print('Accuracy CatBoost:', accuracy)
print('F1 Score CatBoost:', f1_score)
print('ROC AUC CatBoost:', roc_auc)
print('\n')